In this tutorial, we'll take a look at how CrypTen performs inference with an encrypted neural network on encrypted data. We'll see how the data remains encrypted through all the operations, and yet is able to obtain accurate results after the computation. 


### A Simple Linear Layer
We'll start by examining how a single Linear layer works in CrypTen. We'll instantiate a Linear layer, encrypt it, and step through some toy data with it. 

In [1]:
import crypten
import torch

#generate some toy data
features = 4
examples = 3
toy_data = torch.rand(examples, features)

INFO:root:==================
INFO:root:DistributedCommunicator with rank 0
INFO:root:==================
INFO:root:World size = 1


In [2]:
#Instantiate single Linear layer
layer_enc = crypten.nn.Linear(4, 2)

#The weights and the bias are initialized to small random values
print("Plaintext Weights:", layer_enc._parameters['weight'])
print("Plaintext Bias:", layer_enc._parameters['bias'])

Plaintext Weights: Parameter containing:
tensor([[ 0.3743,  0.1923, -0.0274,  0.2341],
        [ 0.1714, -0.2908, -0.1384, -0.4539]], requires_grad=True)
Plaintext Bias: Parameter containing:
tensor([-0.4228, -0.4911], requires_grad=True)


In [3]:
#Encrypt the linear layer 
#TODO: encrypt() needs a src!
layer_enc.encrypt()

#Let's examine the weights and the bias again
#First, we'll see that weight and bias have become MPCTensors
print("Weights Encrypted:", crypten.is_encrypted_tensor(layer_enc._parameters['weight']))
print("Bias Encrypted:", crypten.is_encrypted_tensor(layer_enc._parameters['bias']))
print()

#Now let's look at the tensor values
print("Encrypted Weights:\n", layer_enc._parameters['weight']._tensor._tensor)
print("Encrypted Bias:\n", layer_enc._parameters['bias']._tensor._tensor)

Weights Encrypted: True
Bias Encrypted: True

Encrypted Weights:
 tensor([[ 24533,  12599,  -1797,  15344],
        [ 11230, -19056,  -9073, -29748]])
Encrypted Bias:
 tensor([-27706, -32185])


In [4]:
#Now let's encrypt our data
toy_data_enc = crypten.cryptensor(toy_data)
print("Encrypted data:\n", toy_data_enc._tensor._tensor)
print()

#apply the encrypted layer: encrypted linear transformation 
result_enc = layer_enc.forward(toy_data_enc)
print("Encrypted result:\n", result_enc._tensor._tensor)
print()

#decrypt the result:
print("Decrypted result:\n", result_enc.get_plain_text())

Encrypted data:
 tensor([[57011, 41316, 12289, 22436],
        [10879, 18654,  3557, 59101],
        [63182, 61965, 40347, 50172]])

Encrypted result:
 tensor([[  6494, -46314],
        [ -6308, -63064],
        [ 18498, -67735]])

Decrypted result:
 tensor([[ 0.0991, -0.7067],
        [-0.0963, -0.9623],
        [ 0.2823, -1.0336]])


We can see that the application of the encrypted linear layer on the encrypted data produces an encrypted result, which we can then decrypt to get the values in plaintext.

Let's look at a second linear transformation, to give a flavor of how accuracy is preserved even when the data and the layer are encrypted. We'll look at a uniform scaling transformation, in which all tensor elements are multiplied by the same scalar factor. 

In [5]:
#Initialize a linear layer with random weights
layer_enc = crypten.nn.Linear(3, 3)

#Construct a uniform scaling matrix: we'll scale by factor 5
factor = 5
layer_enc._parameters['weight'] = torch.eye(3)*factor
layer_enc._parameters['bias'] = torch.zeros_like(layer_enc._parameters['bias'])

#Encrypt the layer
layer_enc.encrypt()
print("Encrypted Weights:\n", layer_enc._parameters['weight']._tensor._tensor)
print("Encrypted Bias:\n", layer_enc._parameters['bias']._tensor._tensor)

Encrypted Weights:
 tensor([[327680,      0,      0],
        [     0, 327680,      0],
        [     0,      0, 327680]])
Encrypted Bias:
 tensor([0, 0, 0])


In [6]:
#Construct some toy data
toy_data = torch.ones(2, 3)

#Encrypt the toy data
toy_data_enc = crypten.cryptensor(toy_data)
print("Encrypted data:\n", toy_data_enc._tensor._tensor)

#Apply the encrypted scaling transformation
result_enc = layer_enc.forward(toy_data_enc)
print("Encrypted result:\n", result_enc._tensor._tensor)
print()

print("Plaintext result:\n", result_enc.get_plain_text())

Encrypted data:
 tensor([[65536, 65536, 65536],
        [65536, 65536, 65536]])
Encrypted result:
 tensor([[327680, 327680, 327680],
        [327680, 327680, 327680]])

Plaintext result:
 tensor([[5., 5., 5.],
        [5., 5., 5.]])


The resulting plaintext tensor is correctly scaled, even though we applied the encrypted transformation on the encrypted input! 

### Multi-layer Neural Networks
Let's now look at how the encrypted input moves through an encrypted multi-layer neural network. To keep the explanations simple, we'll use a network with two linear layers and ReLU activations. We'll assume Alice has a private network that Bob wants to run encrypted inference on.

To simulate this, we'll once again generate some toy data and train Alice's network on it. Then we'll encrypt Alice's network, Bob's data, and step through every layer in the network with the encrypted data. Through this, we'll see how the computations get applied although the network and the data are encrypted.

In [7]:
# We'll generate some random data for illustration purposes.
features = 50
examples = 100
data = torch.randn(examples, features)
w_true = torch.randn(1, features)
b_true = torch.randn(1)
y = w_true.matmul(data.t()) + b_true
y = y.sign()

#change labels to format needed by training
y2 = torch.where(y==-1, torch.zeros(y.size()), y)
y2 = y2.squeeze().long()

In [8]:
#Define Alice and Bob's data
data_alice = data[:90,:]
data_bob = data[90:,:]

label_alice = y2[:90]
label_bob = y2[90:]

In [9]:
#Alice creates a very simple one layer network
import torch.nn as nn
import torch.nn.functional as F

#Define Alice's network
class AliceNet(nn.Module):
    def __init__(self):
        super(AliceNet, self).__init__()
        self.fc1 = nn.Linear(50, 20)
        self.fc2 = nn.Linear(20, 2)
        
    def forward(self, x):
        out = self.fc1(x)
        out = F.relu(out)
        out = self.fc2(out)
        return out

model = AliceNet()

#Train Alice's network
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)

for i in range(500):  
    #forward pass: compute prediction
    output = model(data_alice)
    
    #compute and print loss
    loss = criterion(output, label_alice)
    if i % 100 == 99:
        print("Epoch", i, "Loss:", loss.item())
    
    #zero gradients for learnable parameters
    optimizer.zero_grad()
    
    #backward pass: compute gradient with respect to model parameters
    loss.backward()
    
    #update model parameters
    optimizer.step()

Epoch 99 Loss: 0.09137161076068878
Epoch 199 Loss: 0.029487576335668564
Epoch 299 Loss: 0.01661916822195053
Epoch 399 Loss: 0.011747055687010288
Epoch 499 Loss: 0.00933974701911211


In [10]:
#Alice encrypts her network

#Create dummy input of the correct input shape for the model
dummy_input = torch.empty((1, 50))

#Encrypt the network
alice_private_model = crypten.nn.from_pytorch(model, dummy_input)
alice_private_model.encrypt()

#Let's look at what the encrypted network looks like
for name, curr_module in alice_private_model._modules.items():
    print("Name:", name, "\tModule:", curr_module)

Name: 5 	Module: <crypten.nn.module.Linear object at 0x7f9960458278>
Name: 6 	Module: <crypten.nn.module.ReLU object at 0x7f99604582e8>
Name: output 	Module: <crypten.nn.module.Linear object at 0x7f99604582b0>


We see that the encrypted networks has 3 modules, named '5', '6' and 'output', denoting the first Linear layer, the ReLU activation, and the second Linear layer respectively. These modules are encrypted just as the layers in the previous section were. 

Now let's encrypt Bob's data, and step it through each encrypted module. For readability, we will use only 3 examples from Bob's data to illustrate the inference. Note how Bob's data remains encrypted after each individual layer's computation!

In [11]:
#TODO: src=1
data_bob_enc = crypten.cryptensor(data_bob[:3,:])

In [12]:
#forward through the first layer
out_enc = alice_private_model._modules['5'].forward(data_bob_enc)
print("First Linear Layer:\n Output Encrypted:", crypten.is_encrypted_tensor(out_enc))
print(" Tensor values:\n", out_enc._tensor._tensor)

First Linear Layer:
 Output Encrypted: True
 Tensor values:
 tensor([[ -99751,   20290,  -12783,   30416,   -2868,   41344,  -28589,   15760,
          -31889,   22338,   67787,   23732,   68546,   70207,  -45290,   42330,
           49006,  -56520,   23644,  -25871],
        [ -66496,    6791,    9580,   35294,  139194,  -51500,   19774,  -12889,
           17738,    4093,   69535, -119679,   32774,   25008,  -13472,   16878,
          119987,   88731,   83779,  -61285],
        [  81934,   25924,  107973,   29638,   24051,  -25586,   31877,   44093,
           22983,  -46466,   -1503,   34794,     953,  -53280,   39561,  -20057,
           16802,   -8360,   23715,   76248]])


In [13]:
#apply ReLU activation
out_enc = alice_private_model._modules['6'].forward(out_enc)
print("ReLU:\n Output type:", crypten.is_encrypted_tensor(out_enc))
print(" Tensor values:\n", out_enc._tensor._tensor)

ReLU:
 Output type: True
 Tensor values:
 tensor([[     0,  20290,      0,  30416,      0,  41344,      0,  15760,      0,
          22338,  67787,  23732,  68546,  70207,      0,  42330,  49006,      0,
          23644,      0],
        [     0,   6791,   9580,  35294, 139194,      0,  19774,      0,  17738,
           4093,  69535,      0,  32774,  25008,      0,  16878, 119987,  88731,
          83779,      0],
        [ 81934,  25924, 107973,  29638,  24051,      0,  31877,  44093,  22983,
              0,      0,  34794,    953,      0,  39561,      0,  16802,      0,
          23715,  76248]])


In [14]:
#forward through the second Linear layer
out_enc = alice_private_model._modules['output'].forward(out_enc)
print("Second Linear layer:\n Output Encrypted:", crypten.is_encrypted_tensor(out_enc)), 
print(" Tensor values:\n", out_enc._tensor._tensor)

Second Linear layer:
 Output Encrypted: True
 Tensor values:
 tensor([[ -48232,  128517],
        [ 206110, -142978],
        [  -7867,   63898]])


In [15]:
#decrypt the output
out_dec = out_enc.get_plain_text()
print("Decrypted output:\n Output Encrypted:", crypten.is_encrypted_tensor(out_dec))
print(" Tensor values:\n", out_dec)

Decrypted output:
 Output Encrypted: False
 Tensor values:
 tensor([[-0.7360,  1.9610],
        [ 3.1450, -2.1817],
        [-0.1200,  0.9750]])


Again, we emphasize that the output of each layer is an encrypted tensor. Only after the final call to ```get_plain_text``` do we get the plaintext tensor.

We have used a simple two-layer network in our example, but the same ideas apply to more complex networks and operations, and this is what allows CrypTen to successfully perform inference while keeping every computation encrypted. For more details, please refer to the CrypTen whitepaper.